<a href="https://colab.research.google.com/github/ykitaguchi77/GO_AI_project/blob/main/YOLOv8_to_EfficientNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**YOLOv8 to EfficientNetv2 pipeline**

#**Olympia datasetから目を切り抜く（YOLOv8n）**
#**切り抜いた目をEfficientNetV2で解析する**

In [ ]:
"""
Olympia dataset
Dlibで目が2つ検出されるものを抜き出す
YOLOv8を用いて左右とバウンディングボックスを認識させる
切り抜いたバウンディングボックス内の画像について、EfficientNetv2を用いて眼球突出度、MRD-1、MRD-2を回帰させる
スマホに実装
"""

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import shutil
import copy
import pandas as pd
import csv
from tqdm import tqdm

import glob
import random
from PIL import Image
%matplotlib inline

#サポートパッチのインポート
from google.colab.patches import cv2_imshow
import cv2

plt.ion()   # interactive mode

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [77]:
#親フォルダ
parent_dir = '/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv8'

#元画像フォルダ
dataset_dir = '/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset'

#元画像をコピー
orig_dir = f"{parent_dir}/dataset_orig"

#切りぬいた画像を保存するフォルダ
out_dir = f"{parent_dir}/dataset_uni"

#CSVファイルのフォルダ
csv_path = f"{parent_dir}/dataset_uni_for_YOLO8.csv"

In [ ]:
# parent_dirがあれば削除する
if os.path.exists(parent_dir):
    shutil.rmtree(parent_dir)

# 新しくparent_dirを作成する
os.makedirs(parent_dir)

# orig_dir, out_dirを新規に作成する
os.makedirs(orig_dir)
os.makedirs(out_dir)

# orig_dirにdataset_dir直下のファイルをすべてコピーする
file_list = os.listdir(dataset_dir)
for filename in tqdm(file_list, desc="Copying files", unit="file"):
    src_path = os.path.join(dataset_dir, filename)
    dst_path = os.path.join(orig_dir, filename)
    shutil.copy(src_path, dst_path)

print("処理が完了しました。")


Copying files: 100%|██████████| 1016/1016 [00:53<00:00, 19.03file/s]

処理が完了しました。


#**HaarCascadeを用いて目を検出**

In [ ]:
# カスケードファイルのパス
eye_cascade_path = '/content/drive/My Drive/Deep_learning/haarcascade_eye.xml'

# カスケード分類器の特徴量取得
eye_cascade = cv2.CascadeClassifier(eye_cascade_path)


#**目が2つ以上検出されたものを抜き出す**

In [ ]:
with open(csv_path, 'w', newline='') as f:
        #fieldnames = ['Number', 'Folder', 'FileName']
        writer = csv.writer(f)
        writer.writerow(['id','img_path', 'side R/L', 'ex', 'ey', 'ew', 'eh'])  #header

        files = os.listdir(orig_dir)

        k=0
        for file in files:  #フォルダ数の分だけ
              file_path = f"{orig_dir}/{file}"
              id = os.path.splitext(os.path.basename(file_path))[0]

              img = cv2.imread(file_path)
              img2 = img.copy()

              # 画像グレースケール化
              grayscale_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

              #300pix以上のもので目に見えるものを抽出
              eye_list = eye_cascade.detectMultiScale(grayscale_img, minSize=(300, 300))

              # 眼検出判定
              if len(eye_list) >= 1:
                  print('目が' + str(len(eye_list)) +'個検出されました')
              else:
                  print("eye detection error")

              #画像の切り抜きと保存（2個以上検出の時に限る）
              if len(eye_list) >= 2:
                  for (ex, ey, ew, eh) in eye_list:
                      print(f"img_width: {img2.shape[1]}")
                      print("[ex,ey] = %d,%d [ew,eh] = %d,%d" %(ex, ey, ew, eh))
                      cv2.rectangle(img2, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
                      #img_cropped = img[int(ey-0.1*eh): int(ey+1.1*eh), int(ex-0.1*ew): int(ex+1.1*ew)] #本来の切り抜きより幅の0.1倍ずつ水増しする
                      img_cropped = img[int(ey): int(ey+eh), int(ex): int(ex+ew)]


                      if ex+eh*0.5 <= img2.shape[1]/2:
                          side = "R" #横幅の半分より左にあるのは右眼
                      else:
                          side = "L" #横幅の半分よりより右にあるのは左眼

                      print(f"side: {side}")
                      print("")

                      cv2.imwrite(f"{out_dir}/{id}_{side}.png", img_cropped)

                      #対応表の作成
                      writer.writerow([id, file_path, side, ex, ey, ew, eh])
                  else:
                      pass


##**ここで、目以外が誤検出されているものを手動で抜き出して削除する**

In [ ]:
"""
不適切な画像があれば、左右まとめて消す

"""

In [83]:
#CSVのリストと画像フォルダを見比べて、消した画像を判別。CSVを修正する。
import pandas as pd

# Load the CSV
df = pd.read_csv(csv_path)

# Get a list of unique ids
csv_id_list = df['id'].unique().tolist()

print(f"csv_id_list: {csv_id_list}")


# Construct the output directory path
out_dir = f"{parent_dir}/dataset_uni"

# Get a list of all image file names in the directory
image_file_names = os.listdir(out_dir)

# Split each file name at "_" and take the first part, then make sure the list is unique
image_id_list = list(set([int(name.split('_')[0]) for name in image_file_names]))
print(f"image_id_list: {image_id_list}")

# Find the ids that are in the CSV list but not in the image file list
missing_ids = [id for id in csv_id_list if id not in image_id_list]
print(f"missing_ids: {missing_ids}")



# Remove the rows with ids in missing_ids
df = df[~df['id'].isin(missing_ids)]

# Save the dataframe back to a CSV
df.to_csv(f"{parent_dir}/dataset_uni_for_YOLO8_modified.csv", index=False)

csv_id_list: [19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 41, 45, 47, 46, 48, 49, 50, 52, 53, 54, 55, 56, 57, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 89, 88, 90, 91, 92, 93, 94, 95, 96, 97, 936, 99, 101, 100, 102, 103, 104, 105, 106, 107, 109, 112, 110, 111, 113, 115, 116, 937, 117, 118, 120, 121, 124, 123, 125, 122, 938, 126, 127, 128, 129, 939, 131, 133, 132, 134, 135, 137, 136, 940, 139, 138, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 156, 155, 941, 157, 158, 159, 160, 162, 161, 945, 163, 164, 165, 166, 167, 168, 169, 946, 170, 947, 171, 172, 173, 174, 175, 176, 177, 178, 948, 179, 949, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 200, 199, 201, 950, 202, 203, 951, 204, 205, 206, 208, 207, 210, 209, 212, 213, 214, 215, 952, 216, 217, 953, 219, 954, 220, 221, 222, 223, 224, 225, 22

In [ ]:
"""
①"/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv8/dataset_orig"のファイル名（1.jpgとすると1）と、②"/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv8/dataset_uni"のアンダーバーの手前（1_R.jpgとすると1)を見比べる

"/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv8/dataset_dlib_detected"というフォルダを作成
①と②で一致するものに関して、①の画像（コピー）を一辺512pixの黒塗りletterboxにして変換し、新しく作成したフォルダに保存する
"""

from PIL import Image
import os

def letterbox_image(image, size):
    """
    画像を指定したサイズの黒塗りletterboxに変換する関数
    :param image: 変換する画像（PIL Image）
    :param size: 変換後のサイズ（幅, 高さ）のタプル
    :return: 変換後の画像（PIL Image）
    """
    new_width, new_height = size
    old_width, old_height = image.size
    ratio = min(new_width / old_width, new_height / old_height)
    new_width = int(old_width * ratio)
    new_height = int(old_height * ratio)
    resized_image = image.resize((new_width, new_height), Image.LANCZOS)
    boxed_image = Image.new("RGB", size, (0, 0, 0))
    boxed_image.paste(resized_image, ((size[0] - new_width) // 2, (size[1] - new_height) // 2))
    return boxed_image

def main():
    # 保存先フォルダのパス
    output_folder = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv8/dataset_dlib_detected"

    # フォルダが存在しない場合は作成
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)
    os.makedirs(output_folder)

    # ①のフォルダ内のファイル名を取得して、1.jpgのような形式に変換
    orig_folder = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv8/dataset_orig"
    orig_files = os.listdir(orig_folder)
    orig_filenames = [os.path.splitext(filename)[0] for filename in orig_files]

    # ②のフォルダ内のファイル名からアンダーバーの手前を取得して、1_R.jpgのような形式に変換
    uni_folder = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv8/dataset_uni"
    uni_files = os.listdir(uni_folder)
    uni_filenames = [filename.split("_")[0] for filename in uni_files]

    # ①と②で一致するファイルを探して、処理を実行
    orig_set = set(orig_filenames)
    uni_set = set(uni_filenames)

    common_filenames = orig_set.intersection(uni_set)

    print(f"In {len(common_filenames)} images, both eyes were detected. Copy files to the folder 'dataset_dlib_detected'.")

    for filename in tqdm(common_filenames, desc="Processing Images"):
        orig_file_path = os.path.join(orig_folder, filename + ".JPG")
        uni_file_path = os.path.join(uni_folder, filename + "_R.JPG")  # "R"を指定していますが、"L"の場合もあるかもしれません
        output_file_path = os.path.join(output_folder, filename + ".JPG")  # 拡張子を.JPGに変更


        print(f"output_file_path: {output_file_path}")

        # try:
        # ①の画像を読み込み
        orig_image = Image.open(orig_file_path)
        # 一辺512pxの黒塗りletterboxに変換
        new_image = letterbox_image(orig_image, (512, 512))
        # 変換後の画像を保存
        new_image.save(output_file_path)
        # except Exception as e:
        #     print(f"Error: Failed to process image '{orig_file_path}' or save '{output_file_path}'. Skipping.")

if __name__ == "__main__":
    main()


In [84]:
num = os.listdir("/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv8/dataset_dlib_detected")
print(len(num))

958


#**YOLOv8 annotations**

In [85]:
#親フォルダ
parent_dir = '/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv8'

#元画像フォルダ
dataset_dir = '/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset'

#元画像をコピー
orig_dir = f"{parent_dir}/dataset_orig"

#切りぬいた画像を保存するフォルダ
out_dir = f"{parent_dir}/dataset_uni"

#CSVファイルのフォルダ
csv_path = f"{parent_dir}/dataset_uni_for_YOLO8_modified.csv"

In [88]:
# CSVファイルを読み込んでDataFrameに格納
df = pd.read_csv(csv_path)

# DataFrameの内容を表示
df

,id,img_path,side R/L,ex,ey,ew,eh
0,19,/content/drive/MyDrive/Deep_learning/Olympia_d...,R,177,627,696,696
1,19,/content/drive/MyDrive/Deep_learning/Olympia_d...,L,1584,618,708,708
2,20,/content/drive/MyDrive/Deep_learning/Olympia_d...,L,1570,568,815,815
3,20,/content/drive/MyDrive/Deep_learning/Olympia_d...,R,152,561,785,785
4,21,/content/drive/MyDrive/Deep_learning/Olympia_d...,L,1636,610,757,757
...,...,...,...,...,...,...,...
1917,15,/content/drive/MyDrive/Deep_learning/Olympia_d...,L,1533,422,767,767
1918,16,/content/drive/MyDrive/Deep_learning/Olympia_d...,L,1548,609,780,780
1919,16,/content/drive/MyDrive/Deep_learning/Olympia_d...,R,178,652,736,736
1920,17,/content/drive/MyDrive/Deep_learning/Olympia_d...,R,216,436,777,777


In [87]:
len(df)

1922

In [103]:
detected_folder = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv8/dataset_dlib_detected"
detected_files = os.listdir(detected_folder)
detected_filenames = [os.path.splitext(filename)[0] for filename in detected_files]

df_ids = df["id"].astype(str)

# detected_filenamesには含まれるけれどDataFrameには見当たらないファイルを探索
for detected_filename in detected_filenames:
    if detected_filename not in df_ids.values:
        print(f"Warning: File '{detected_filename}.JPG' exists in '{detected_folder}' but is not found in the DataFrame.")

# DataFrameには含まれるけれどdetected_filenamesには見当たらないファイルを探索
for df_id in df_ids:
    if df_id not in detected_filenames:
        print(f"Warning: File '{df_id}.JPG' is found in the DataFrame but does not exist in '{detected_folder}'.")

print(len(df_ids))
print(len(detected_filenames))

1922
958


In [ ]:
print(detected_filenames)

In [ ]:
print(df["id"])

In [38]:
from PIL import Image
import os
import pandas as pd


def xywh_to_yolo(class_label, img_width, img_height, x, y, w, h):
    x_center = x / img_width
    y_center = y / img_height
    width = w / img_width
    height = h / img_height

    return f"{class_label} {x_center} {y_center} {width} {height}"

def create_yolo_txt(df):
    grouped_data = df.groupby('id')

    for group_id, group_df in grouped_data:
        txt_file_path = os.path.join(os.path.dirname(group_df.iloc[0]['img_path']), f"{group_id}.txt")

        with open(txt_file_path, 'w') as txt_file:
            for _, row in group_df.iterrows():
                img_path = row['img_path']
                class_label = row['side R/L']
                x, y, w, h = row['ex'], row['ey'], row['ew'], row['eh']

                try:
                    img = Image.open(img_path)
                    img_width, img_height = img.size
                except Exception as e:
                    print(f"Error: Failed to open '{img_path}' or get image dimensions. Skipping annotation.")
                    continue

                # YOLO形式に変換
                yolo_annotation = xywh_to_yolo(class_label, img_width, img_height, x, y, w, h)

                # ファイルに書き込み
                yolo_annotation += '\n'  # 改行を追加
                txt_file.write(yolo_annotation)        # ファイル内容を参照する

        with open(txt_file_path, 'r') as txt_file_read:
            file_contents = txt_file_read.read()
            print(f"Contents of '{txt_file_path}':\n{file_contents}")

# YOLO形式のテキストファイルを作成して保存
df_sample = df[0:6]
create_yolo_txt(df_sample)


Contents of '/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv8/dataset_orig/19.txt':
R 0.06828703703703703 0.3628472222222222 0.26851851851851855 0.4027777777777778
L 0.6111111111111112 0.3576388888888889 0.27314814814814814 0.4097222222222222

Contents of '/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv8/dataset_orig/20.txt':
L 0.6057098765432098 0.3287037037037037 0.314429012345679 0.47164351851851855
R 0.05864197530864197 0.3246527777777778 0.3028549382716049 0.4542824074074074

Contents of '/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_uni_for_YOLOv8/dataset_orig/21.txt':
L 0.6311728395061729 0.35300925925925924 0.29205246913580246 0.4380787037037037
R 0.08449074074074074 0.3628472222222222 0.2789351851851852 0.4184027777777778



#**YOLOv8 interference**

In [2]:
# YOLOv8 setup
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.142 🚀 Python-3.10.6 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.3/225.8 GB disk)


In [ ]:
!wget /content https://pds.exblog.jp/logo/1/197001/01/18/b043001820221113221721.jpg
!wget /content https://plugins-media.makeupar.com/smb/blog/post/2023-06-15/37011ea9-289b-4700-9e61-6a1429128ae6.jpg
!wget /content https://www.city.gosen.lg.jp/material/images/group/5/chugatasu.JPG
img_path = ["/content/b043001820221113221721.jpg", "37011ea9-289b-4700-9e61-6a1429128ae6.jpg", "chugatasu.JPG"]

In [ ]:
import os
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as DisplayImage
from IPython.display import display
import cv2
# 事前学習済みのモデルを読み込み(detectionモデルを使用)
model = YOLO('yolov8n.pt')
# predictモードを実行 (結果だけ欲しいので、project・name・exist_okはなくてもOK)
results = model.predict(source=img_path[2])
# Resultsオブジェクトから描画に必要な情報を取得
coordinate_bbox = results[0].boxes.xyxy #bbox
classes=results[0].boxes.cls # 検出クラス
classes_map = results[0].names # クラス番号と名称
# 画像の読み込み
img = Image.open(results[0].path)
# 色の指定 (クラスごとにランダムに色を選択する場合使う)
# colors = ['red', 'green', 'blue', 'yellow', 'cyan', 'magenta', 'olive', 'purple']
# colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255), (255, 0, 255), (128, 128, 0), (128, 0, 128)]
# 描画コンテントの取得
draw = ImageDraw.Draw(img)
"""
フォントの設定。Macだと/System/Library/Fontsに色々あるのでここから選んだ。
bboxだけの描画であれば必要ない。必要ない場合はdraw.text(font=font)のfont部分を消す。
エラーが出る場合はフォントファイルへのフルパス(/System/Library/Fonts/ヒラギノ丸ゴ ProN W4.ttc)で設定してあげる
"""
font = ImageFont.truetype('/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf', 4000)
# バウンディングボックスの描画
i=1
for bbox, cls in zip(coordinate_bbox, classes):
    x1, y1, x2, y2 = map(int, bbox)
    #color = colors[int(cls) % len(colors)] 今回は色を直接指定するので使わない
    color=(255,0,0) # red
    # bboxの描画
    draw.rectangle([x1, y1, x2, y2], outline=color, width=5)
    cls_text = classes_map.get(int(cls))
    # 検出クラス名の描画
    draw.text((x1, y1 - 50), cls_text+"#"+str(i), fill="orange")
    print(f"class: {cls_text}")
    print(f"x1: {x1}, y1: {y1}, x2: {x2}, y2: {y2}")
    i += 1
# 画像のリサイズ (必要であれば)
#img = img.resize((640,640))
# 出力先フォルダの作成。model.predictで作成させてもOK。
directories = ['/Users/hinomaruc/Desktop/blog/dataset/yolov8/runs/mypredict']
for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)
# 画像の保存 (存在しないフォルダだとFileNotFoundErrorになるので注意)
img.save('/Users/hinomaruc/Desktop/blog/dataset/yolov8/runs/mypredict/sample_bbox.png')
# 画像の表示
display(DisplayImage(filename='/Users/hinomaruc/Desktop/blog/dataset/yolov8/runs/mypredict/sample_bbox.png'))

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import os
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as DisplayImage
from IPython.display import display

# 事前学習済みのモデルを読み込み(detectionモデルを使用)
model = YOLO('yolov8n.pt')
# predictモードを実行 (結果だけ欲しいので、project・name・exist_okはなくてもOK)
results = model.predict(source=img_path[1])
# Resultsオブジェクトから描画に必要な情報を取得
coordinate_bbox = results[0].boxes.xyxy #bbox
classes=results[0].boxes.cls # 検出クラス
classes_map = results[0].names # クラス番号と名称

# Open the image file
img_cv2 = cv2.imread(results[0].path)

# Set the font scale and thickness
font_scale = 3
font_thickness = 2

# Loop through each bounding box
i=1
for bbox, cls in zip(coordinate_bbox, classes):
    print(bbox)

    x1, y1, x2, y2 = map(int, bbox)
    color=(0,0,255) # red
    # Draw the bounding box
    cv2.rectangle(img_cv2, (x1, y1), (x2, y2), color, 5)
    cls_text = classes_map.get(int(cls))
    # Put the class label text
    cv2.putText(img_cv2, cls_text+"#"+str(i), (x1, y1 - 50), cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, font_thickness)
    i += 1

# Save the image
cv2.imwrite('/Users/hinomaruc/Desktop/blog/dataset/yolov8/runs/mypredict/sample_bbox.png', img_cv2)


#plt.imshow(cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB))
# Convert the color from BGR to RGB
img_rgb = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)

# Use matplotlib to display the image
plt.imshow(img_rgb)

# Show the axes
plt.axis('on')

# Show the image
plt.show()
